<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP2/C%2B%2B/TP1_P2_M2_Colab_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizo el comando writefile para crear un archivo llamado tp1p2.cpp. Necesitaré ese archivo para crear un ejecutable.

In [7]:
%%writefile tp1p2.cpp
#include <iostream>
#include <string>
#include <thread>
#include <cstdlib>
#include <ctime>
#include <iomanip>
#include <vector>
#include <ctype.h>
#include <cstring>

#define PARAMETROS_INCORRECTOS 0
#define N_FUERA_DE_RANGO PARAMETROS_INCORRECTOS
#define PARAMETRO_NO_ENTERO PARAMETROS_INCORRECTOS
#define ESPACIO_CARACTERES 6
#define INICIO_ITERADOR 0
#define PARAMETROS_CORRECTOS 2
#define RANGO_MINIMO 5
#define RANGO_MAXIMO 20
#define VALOR_ALEATORIO_MAXIMO 65
#define VALOR_ALEATORIO_NEGATIVO 32
#define EXITO 1
#define INICIALIZADOR 0

struct MatricesHilos
{
  int** matriz_a;
  int** matriz_b;
  int** matriz_cs;
  int** matriz_ch;
  int tamanio_matriz;
  int cantidad_hilos;
};

bool CantidadCorrectaDeParametros(int parametros_recibidos, int parametros_esperados);
bool ParametroNEsEntero(char* parametro);
bool RangoValidoN(int valor, int valor_minimo, int valor_maximo);
int** CrearMatriz(int tamanio_matriz);
void MostrarMatriz(int **matriz,int tamanio_matriz,const char* nombre);
MatricesHilos LlenarEstructura(int tamanio);
void LlenarMatricesAleatorio();
void MultiplicarMatricesSecuencial();
void MultiplicarMatricesConHilos(int cantidad_hilos);
void MostrarMatrices();
void FuncionEjecutadaPorHilo(int numero_hilo);
void CompararMatricesResultado();

MatricesHilos matriz_hilos;

int main(int argc, char* argv[])
{
  if(!CantidadCorrectaDeParametros(argc,PARAMETROS_CORRECTOS))
  {
    std::cout<<"No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo"<<std::endl;
    return PARAMETROS_INCORRECTOS;
  }
  char* parametro_n = argv[1];
  if (!ParametroNEsEntero(parametro_n))
  {
    std::cout<<"El parametro que usted ingreso no es un entero positivo"<<std::endl;
    return PARAMETRO_NO_ENTERO;
  }
  int tamanio_matrices = atoi(argv[1]);
  if (!RangoValidoN(tamanio_matrices,RANGO_MINIMO,RANGO_MAXIMO))
  {
    std::cout<<"El rango de N debe estar entre 5 y 20"<<std::endl;
    return N_FUERA_DE_RANGO;
  }
  
  matriz_hilos = LlenarEstructura(tamanio_matrices);
  LlenarMatricesAleatorio();
  MultiplicarMatricesSecuencial();
  MultiplicarMatricesConHilos(matriz_hilos.cantidad_hilos);
  MostrarMatrices();
  CompararMatricesResultado();
  return EXITO;
}

bool CantidadCorrectaDeParametros(int parametros_recibidos, int parametros_esperados)
{
  return parametros_recibidos == parametros_esperados;
}

bool ParametroNEsEntero(char* parametro)
{
  for (int i = INICIO_ITERADOR; i < strlen(parametro); i++)
  {
    if (!isdigit(parametro[i]))
    {
      return false;
    }
  }
  return true;
}

bool RangoValidoN(int valor, int valor_minimo, int valor_maximo)
{
  return valor >= valor_minimo && valor <= valor_maximo;
}

int** CrearMatriz(int tamanio_matriz)
{
  int** matriz = new int*[tamanio_matriz];
  for (int i = INICIO_ITERADOR; i < tamanio_matriz; i++)
  {
    matriz[i] = new int[tamanio_matriz];
    for (int j = INICIO_ITERADOR; j < tamanio_matriz; j++)
    {
      matriz[i][j] = INICIALIZADOR;
    }
  }
  return matriz;
}

void MostrarMatriz(int **matriz,int tamanio_matriz,const char* nombre)
{
  std::cout<<nombre<<std::endl;
  for (int i = INICIO_ITERADOR; i < tamanio_matriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < tamanio_matriz; j++)
    {
      std::cout << std::setw(ESPACIO_CARACTERES) << matriz[i][j];
    }
    std::cout << std::endl;
  }
  std::cout << std::endl;
}

MatricesHilos LlenarEstructura(int tamanio)
{
  MatricesHilos matriz_hilo;
  matriz_hilo.matriz_a = CrearMatriz(tamanio);
  matriz_hilo.matriz_b = CrearMatriz(tamanio);
  matriz_hilo.matriz_cs = CrearMatriz(tamanio);
  matriz_hilo.matriz_ch = CrearMatriz(tamanio);    
  matriz_hilo.tamanio_matriz = matriz_hilo.cantidad_hilos = tamanio;
  return matriz_hilo;
}

void LlenarMatricesAleatorio()
{
  srand(time(NULL));
  for (int i = INICIO_ITERADOR; i < matriz_hilos.tamanio_matriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matriz_hilos.tamanio_matriz; j++)
    {
      matriz_hilos.matriz_a[i][j] = rand() % VALOR_ALEATORIO_MAXIMO - VALOR_ALEATORIO_NEGATIVO;
      matriz_hilos.matriz_b[i][j] = rand() % VALOR_ALEATORIO_MAXIMO - VALOR_ALEATORIO_NEGATIVO;
    }
  }    
}

void MultiplicarMatricesSecuencial()
{
  for (int i = INICIO_ITERADOR; i < matriz_hilos.tamanio_matriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matriz_hilos.tamanio_matriz; j++)
    {
      matriz_hilos.matriz_cs[i][j] = INICIALIZADOR;
      for (int k = INICIO_ITERADOR; k < matriz_hilos.tamanio_matriz; k++)
      {
        matriz_hilos.matriz_cs[i][j] += matriz_hilos.matriz_a[i][k] * matriz_hilos.matriz_b[k][j];
      }
    }
  }    
}

void MultiplicarMatricesConHilos(int cantidad_hilos)
{
  std::vector<std::thread> hilos;
  for (int i = INICIO_ITERADOR; i < cantidad_hilos; i++)
  {
    hilos.emplace_back(FuncionEjecutadaPorHilo,i);
  }
  for (auto& hilo : hilos) 
  {
    hilo.join();
  }
}

void MostrarMatrices()
{
  MostrarMatriz(matriz_hilos.matriz_a,matriz_hilos.tamanio_matriz,"Matriz A");
  MostrarMatriz(matriz_hilos.matriz_b,matriz_hilos.tamanio_matriz,"Matriz B");
  MostrarMatriz(matriz_hilos.matriz_cs,matriz_hilos.tamanio_matriz,"Matriz CS");
  MostrarMatriz(matriz_hilos.matriz_ch,matriz_hilos.tamanio_matriz,"Matriz CH");
}

void FuncionEjecutadaPorHilo(int numero_hilo)
{
  for (int j = INICIO_ITERADOR; j < matriz_hilos.tamanio_matriz; j++)
  {
    for (int k = INICIO_ITERADOR; k < matriz_hilos.tamanio_matriz; k++)
    {
      matriz_hilos.matriz_ch[numero_hilo][j] += matriz_hilos.matriz_a[numero_hilo][k] * matriz_hilos.matriz_b[k][j];
    }
  }
}

void CompararMatricesResultado()
{
  bool son_iguales = true;
  for (int i = INICIO_ITERADOR; i < matriz_hilos.tamanio_matriz; i++)
  {
    for (int j = INICIO_ITERADOR; j < matriz_hilos.tamanio_matriz; j++)
    {
      if (matriz_hilos.matriz_cs[i][j] != matriz_hilos.matriz_ch[i][j])
      {
        son_iguales = false;
        break;
      }
    }
  }
  if (son_iguales)
    std::cout<<"El producto matricial coincide"<<std::endl;
  else
    std::cout<<"El producto matricial no coincide"<<std::endl;
}

Overwriting tp1p2.cpp


Listo los archivos para asegurarme que existe el nuevo archivo que cree (tp1p2.cpp).

In [8]:
!ls -la

total 84
drwxr-xr-x 1 root root  4096 May  1 13:30 .
drwxr-xr-x 1 root root  4096 May  1 13:29 ..
drwxr-xr-x 4 root root  4096 Apr 27 13:34 .config
-rw-r--r-- 1 root root   693 May  1 13:30 salidaC
drwxr-xr-x 1 root root  4096 Apr 27 13:35 sample_data
-rwxr-xr-x 1 root root 55304 May  1 13:30 tp1p2
-rw-r--r-- 1 root root  6034 May  1 13:36 tp1p2.cpp


Compilo el archivo tp1p2.cpp para obtener un ejecutable. En el codigo no utilizo la biblioteca pthread.h ni ninguna otra similar. Sin embargo, es necesario incluirla en el comando para crear el ejecutable. De lo contrario, arroja un error. 

Puede ver más detalles del error que arrojaría si no incluye el ptrhead en [este sitio](https://itslinuxfoss.com/undefined-reference-pthread-create/#:~:text=The%20error%20%E2%80%9Cundefined%20reference%20to,line%20while%20compiling%20the%20code.)

In [9]:
!g++ -std=c++11 -pthread -o tp1p2 tp1p2.cpp

Listo los archivos de nuevo para asegurarme que poseo el ejecutable. Debería salir como tp1p2 sin ninguna clase de extensión.

In [10]:
!ls -la

total 84
drwxr-xr-x 1 root root  4096 May  1 13:36 .
drwxr-xr-x 1 root root  4096 May  1 13:29 ..
drwxr-xr-x 4 root root  4096 Apr 27 13:34 .config
-rw-r--r-- 1 root root   693 May  1 13:30 salidaC
drwxr-xr-x 1 root root  4096 Apr 27 13:35 sample_data
-rwxr-xr-x 1 root root 55304 May  1 13:36 tp1p2
-rw-r--r-- 1 root root  6034 May  1 13:36 tp1p2.cpp


Utilizo el comando nohup para ejecutar el proceso en segundo plano, de manera que no me ocupe la consola. Es necesario añadir el caracter '&' al final.

Para ejecutar el proceso, tengo que poner ./tp1p2 N donde N es un número comprendido en el rango de 5 a 20. 

El comando 1>salidaC va a redirigir la salida estandar de la pantalla (stdout) a un archivo llamado salidaC. 

El comando 2>/dev/null envía la salida de error estandar (stderr) a dev/null

In [11]:
!nohup ./tp1p2 5 1>salidaC 2>/dev/null & 

Reviso el archivo de salida. Debo mostrar la matriz A, la matriz B, la matriz CS (resultado de realizar el producto matricial de forma secuencial), la matriz CH (resultado de realizar el producto matricial utilizando hilos), y por último una línea de texto que me indique si las matrices resultantes son iguales o diferentes. 

In [12]:
!cat salidaC

Matriz A
    16     4   -30   -13    28
    25   -30   -24    -1    12
   -22    -4     7     8    -5
    28    27    -3    21     0
     4    29    32   -25   -29

Matriz B
   -19   -14   -13     1    19
   -22     5    -1    30    21
   -30    16   -14    -4    14
   -22     2    32    18    16
    -8     7   -26    -2    19

Matriz CS
   570  -514  -936   -34   292
   831  -802  -303  -821  -279
   160   381   578   -16  -371
 -1498  -263   323  1228  1393
  -892   348  -575   354   182

Matriz CH
   570  -514  -936   -34   292
   831  -802  -303  -821  -279
   160   381   578   -16  -371
 -1498  -263   323  1228  1393
  -892   348  -575   354   182

El producto matricial coincide


Vuelvo a ejecutar el proceso. Esta vez pasándole 2 parámetros. Redirigiré la salida al archivo salidaExcesoParametros.

In [13]:
!nohup ./tp1p2 5 2 1>salidaExcesoParametros 2>/dev/null & 

Muestro el archivo salidaExcesoParametros. Debe informarme que solo debo ingresar un el valor de N como parámetro. 

In [14]:
!cat salidaExcesoParametros

No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo


Vuelvo a ejecutar el proceso. Esta vez pasándole 0 parámetros. Redirigiré la salida al archivo salidaSinParametros.

In [15]:
!nohup ./tp1p2 1>salidaSinParametros 2>/dev/null & 

Muestro el archivo salidaSinParametros. Debe informarme que solo debo ingresar un el valor de N como parámetro. 

In [16]:
!cat salidaSinParametros

No ingreso la cantidad correcta de parametros. Solamente debe ingresar un entero positivo


Vuelvo a ejecutar el proceso. Esta vez pasándole un valor mayor a 20. Redirigiré la salida al archivo salidaRangoSuperiorExcedido.

In [17]:
!nohup ./tp1p2 25 1>salidaRangoSuperiorExcedido 2>/dev/null & 

Muestro el archivo salidaRangoSuperiorExcedido. Debe informarme que debo ingresar un número entre 5 y 20 como parámetro. 

In [18]:
!cat salidaRangoSuperiorExcedido

El rango de N debe estar entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasándole un valor menor a 5. Redirigiré la salida al archivo salidaRangoInferiorExcedido.

In [19]:
!nohup ./tp1p2 3 1>salidaRangoInferiorExcedido 2>/dev/null & 

Muestro el archivo salidaRangoInferiorExcedido. Debe informarme que debo ingresar un número entre 5 y 20 como parámetro. 

In [20]:
!cat salidaRangoInferiorExcedido

El rango de N debe estar entre 5 y 20


Vuelvo a ejecutar el proceso. Esta vez pasandole un numero no entero. Redirigiré la salida al archivo salidaNumeroNoEntero.

In [21]:
!nohup ./tp1p2 3.1 1>salidaNumeroNoEntero 2>/dev/null & 

Muestro el archivo salidaNumeroNoEntero. Debe informarme que no ingresé un número entero positivo. 

In [22]:
!cat salidaNumeroNoEntero

El parametro que usted ingreso no es un entero positivo
